In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
os.environ['KAGGLE.CONFIG.DIR']="."
#Configuro el directorio porque en mi computadora no encontraba el archivo kaggle.json

In [3]:
from google.colab import files
!mkdir -p ~/.config/kaggle
!mv kaggle.json ~/.config/kaggle/
!chmod 600 ~/.config/kaggle/kaggle.json

In [4]:
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia

  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 1.33GB/s]


In [5]:
!unzip udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip

Archive:  udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               


In [6]:
entrenamiento=pd.read_csv("train.csv")
testeo=pd.read_csv("test.csv")

In [7]:
#Usando LabelEncoder para transformar los valores ordinales del rendimiento a valores numericos.
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
entrenamiento['RENDIMIENTO_NUM'] = le.fit_transform(entrenamiento['RENDIMIENTO_GLOBAL'])

#Escojo las columnas de interés para entrenar el modelo
columnas_seleccionadas = [
    'F_EDUCACIONPADRE',
    'F_EDUCACIONMADRE',
    'F_ESTRATOVIVIENDA',
    'INDICADOR_1',
    'INDICADOR_2',
    'F_TIENEINTERNET'
]

columnas_numericas = ['INDICADOR_1', 'INDICADOR_2']  # Estas son numéricas
columnas_categoricas = [col for col in columnas_seleccionadas if col not in columnas_numericas]

In [8]:
faltantes_tr = entrenamiento[columnas_seleccionadas].isnull().sum()
faltantes_ts = testeo[columnas_seleccionadas].isnull().sum()
print(faltantes_tr)
print(faltantes_ts)


F_EDUCACIONPADRE     23178
F_EDUCACIONMADRE     23664
F_ESTRATOVIVIENDA    32137
INDICADOR_1              0
INDICADOR_2              0
F_TIENEINTERNET      26629
dtype: int64
F_EDUCACIONPADRE      9993
F_EDUCACIONMADRE     10223
F_ESTRATOVIVIENDA    13795
INDICADOR_1              0
INDICADOR_2              0
F_TIENEINTERNET      11539
dtype: int64


In [9]:
# Rellenamos los valores faltantes de las columnas númericas con la mediana
entrenamiento['INDICADOR_1'].fillna(entrenamiento['INDICADOR_1'].median(), inplace=True)
entrenamiento['INDICADOR_2'].fillna(entrenamiento['INDICADOR_2'].median(), inplace=True)
testeo['INDICADOR_1'].fillna(testeo['INDICADOR_1'].median(), inplace=True)
testeo['INDICADOR_2'].fillna(testeo['INDICADOR_2'].median(), inplace=True)

#Y de las categóricas con la moda
for col in columnas_categoricas:
    mode_val = entrenamiento[col].mode()[0]
    entrenamiento[col].fillna(mode_val, inplace=True)
    testeo[col].fillna(mode_val, inplace=True)

/tmp/ipython-input-594081158.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  entrenamiento['INDICADOR_1'].fillna(entrenamiento['INDICADOR_1'].median(), inplace=True)
/tmp/ipython-input-594081158.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col

In [10]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Definir preprocessor, esto es para transformar las columnas númericas en escalares estándar y las columnas categóricas en one-hot
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), columnas_numericas),
        ('cat', OneHotEncoder(drop='first'), columnas_categoricas)
    ])

X_tr_procesado = preprocessor.fit_transform(entrenamiento[columnas_seleccionadas])
X_ts_procesado = preprocessor.transform(testeo[columnas_seleccionadas])

y_tr = entrenamiento['RENDIMIENTO_NUM']  # Usar la versión numérica codificada
print(f"Shapes: X_tr {X_tr_procesado.shape}, y_tr {y_tr.shape}, X_ts {X_ts_procesado.shape}")

Shapes: X_tr (692500, 31), y_tr (692500,), X_ts (296786, 31)


In [11]:
#Entreno el modelo que escogí
from sklearn.ensemble import RandomForestClassifier

rf_model =RandomForestClassifier(
    n_estimators=93,
    max_depth=17,
    min_samples_split=35,
    min_samples_leaf=22,
    n_jobs=-1,
    random_state=42
)

rf_model.fit(X_tr_procesado, y_tr)
predic=rf_model.predict(X_ts_procesado)

In [12]:
y_pred_original = le.inverse_transform(predic)
submission = pd.DataFrame([testeo.ID, pd.Series(y_pred_original, name="RENDIMIENTO_GLOBAL")]).T
submission.shape

(296786, 2)

In [14]:
submission.to_csv("Kaggle_Icfes_Competition.csv", index=False)

In [15]:
!head Kaggle_Icfes_Competition.csv

ID,RENDIMIENTO_GLOBAL
550236,alto
98545,medio-alto
499179,medio-alto
782980,bajo
785185,medio-alto
58495,bajo
705444,alto
557548,alto
519909,bajo
